In [1]:
import sys
sys.path.append('..')
from data.data_reader import *

In [2]:
import tqdm
import scanpy as sc

import pandas as pd

In [3]:
download_file('https://plus.figshare.com/ndownloader/files/35775512','35775512.h5ad')
adata_orig = sc.read_h5ad("35775512.h5ad")
adata_orig.X[adata_orig.X == float("inf")]=0

File downloaded successfully to 35775512.h5ad


In [4]:
adata_orig.obs['gene_name']=list(pd.Series(adata_orig.obs.index).apply(lambda x:x.split("_")[1]))
adata_orig.obs['id']=range(adata_orig.obs.shape[0])

In [5]:
import tqdm
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [6]:
def cosine_similarity(A):
  AAt=np.matmul(A,A.transpose())
  n_A=np.sqrt((A**2).sum(axis=1)).reshape(-1,1)
  n_A=np.matmul(n_A,n_A.transpose())
  return AAt/(n_A)

In [7]:
class X_dataset(Dataset):
    def __init__(self,data):
        self.data=data
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        return {'x':torch.tensor(self.data.X[idx]),'c':torch.tensor(self.data.obs.iloc[idx]['core_control'])}


In [8]:
dataset=X_dataset(adata_orig)
train_loader=DataLoader(dataset,batch_size=32,shuffle=True)

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class Encoder(nn.Module):
    def __init__(self, latent_dim=10,kl_coef=0.000001):
        super(Encoder, self).__init__()
        self.latent_dim=latent_dim
        self.kl_coef=kl_coef
        self.dense1=nn.Linear(dataset[0]['x'].shape[0],1024)
        self.bn1=nn.BatchNorm1d(1024)
        self.dense2=nn.Linear(1024,256)
        self.bn2=nn.BatchNorm1d(256)
        self.mu=nn.Linear(256, latent_dim)
        self.logvar=nn.Linear(256, latent_dim)
        self.kl = 0
    def reparameterize(self, mu , logvar):
        std = torch.exp(logvar*0.5)
        eps = torch.randn_like(std).to(device)
        z = mu + eps * std
        return z
    def forward(self, x):
        bn=x.size(0)
        x=F.relu(self.bn1(self.dense1(x)))
        x=F.relu(self.bn2(self.dense2(x)))
        mu =  self.mu(x)
        logvar = self.logvar(x)
        z=self.reparameterize(mu , logvar)
        self.kl = 0.5*(logvar.exp() + mu**2 - logvar - 1).sum()*self.kl_coef
        return z


class Decoder(nn.Module):
    def __init__(self, latent_dim=8):
        super(Decoder, self).__init__()
        self.dense1=nn.Linear(latent_dim,256)
        self.bn1=nn.BatchNorm1d(256)
        self.dense2=nn.Linear(256,1024)
        self.bn2=nn.BatchNorm1d(1024)
        self.out=nn.Linear(1024,dataset[0]['x'].shape[0])

    def forward(self, z):
        z = F.relu(self.bn1(self.dense1(z)))
        z = F.relu(self.bn2(self.dense2(z)))
        z = self.out(z)
        return z

class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims=10,kl_coef=0.000001):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = Encoder(latent_dims,kl_coef).to(device)
        self.decoder = Decoder(latent_dims).to(device)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)


In [10]:
# autoencoder=VariationalAutoencoder(20,0.1)
autoencoder=VariationalAutoencoder(20,1e-9)
opt = torch.optim.Adam(autoencoder.parameters(),lr=0.001)
loss_fn=torch.nn.MSELoss()


C:\Users\khoda\AppData\Local\Temp\ipykernel_23824\3597674635.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return {'x':torch.tensor(self.data.X[idx]),'c':torch.tensor(self.data.obs.iloc[idx]['core_control'])}


In [11]:
autoencoder.train()
for epoch in range(100):
    train_loss1=0
    train_loss2=0
    autoencoder.train()
    for batch in tqdm.tqdm(train_loader):
        x = batch['x'].to(device) # GPU
        c = batch['c'].to(device) # GPU
        opt.zero_grad()
        x_hat = autoencoder(x)
        loss1=loss_fn(x_hat,x)
        loss2=autoencoder.encoder.kl
        loss = loss1 + loss2
        train_loss1+=loss1.detach().cpu().numpy()
        train_loss2+=loss2.detach().cpu().numpy()
        loss.backward()
        opt.step()
    print(f"TRAIN: EPOCH {epoch}: MSE: {train_loss1/len(train_loader)}, KL_LOSS: {train_loss2/len(train_loader)}")



  0%|          | 0/84 [00:00<?, ?it/s]C:\Users\khoda\AppData\Local\Temp\ipykernel_23824\3597674635.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return {'x':torch.tensor(self.data.X[idx]),'c':torch.tensor(self.data.obs.iloc[idx]['core_control'])}
100%|██████████| 84/84 [00:01<00:00, 56.53it/s]


TRAIN: EPOCH 0: MSE: 0.07599483567866541, KL_LOSS: 1.2950634110421247e-05


100%|██████████| 84/84 [00:00<00:00, 99.57it/s] 


TRAIN: EPOCH 1: MSE: 0.0530475355862152, KL_LOSS: 1.1833086121203537e-05


100%|██████████| 84/84 [00:00<00:00, 93.47it/s] 


...

TRAIN: EPOCH 97: MSE: 0.022693345611471506, KL_LOSS: 5.882818716903178e-06


100%|██████████| 84/84 [00:00<00:00, 95.93it/s]


TRAIN: EPOCH 98: MSE: 0.02128689396860344, KL_LOSS: 5.8265818822855225e-06


100%|██████████| 84/84 [00:00<00:00, 95.55it/s] 

TRAIN: EPOCH 99: MSE: 0.02129365716661726, KL_LOSS: 5.842922730283005e-06


In [12]:
autoencoder.eval()
encoded_x=[]
cs=[]
for rec in tqdm.tqdm(dataset):
    x = rec['x'].reshape(1,-1).to(device) # GPU
    c = rec['c'].reshape(1,).to(device) # GPU
    encoded_x.append(autoencoder.encoder(x).cpu().detach().numpy())
    cs.append(c.cpu().detach().numpy())
encoded_x=np.concatenate(encoded_x,axis=0)
encoded_x=(encoded_x-encoded_x.mean(axis=0,keepdims=True))/encoded_x.std(axis=0,keepdims=True)
cs=np.concatenate(cs,axis=0)
df_to_be_shown=pd.DataFrame(encoded_x,columns=[f'f{i}' for i in range(encoded_x.shape[1])])
df_to_be_shown['control']=cs

  0%|          | 0/2679 [00:00<?, ?it/s]C:\Users\khoda\AppData\Local\Temp\ipykernel_23824\3597674635.py:7: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  return {'x':torch.tensor(self.data.X[idx]),'c':torch.tensor(self.data.obs.iloc[idx]['core_control'])}
100%|██████████| 2679/2679 [00:02<00:00, 1066.78it/s]


In [13]:
cos_sim_f=cosine_similarity(np.array(df_to_be_shown.drop(['control'], axis=1)))

In [14]:
similarity_matrix=np.zeros(cos_sim_f.shape)
similarity_db=hu_data_loader()

for gene_name in tqdm.tqdm(adata_orig.obs.gene_name.unique()):
    query=query_hu_data(similarity_db,gene_name)
    for q in query:
        if q in adata_orig.obs.gene_name.values:
            y_indices=adata_orig.obs[adata_orig.obs.gene_name==q].id
            x_indices=adata_orig.obs[adata_orig.obs.gene_name==gene_name].id
            for x_id in x_indices:
                for y_id in y_indices:
                    similarity_matrix[y_id,x_id]=1
                    similarity_matrix[x_id,y_id]=1

cos_sim_f_flatten=cos_sim_f.reshape(-1,)
similarity_matrix_flatten=similarity_matrix.reshape(-1,)
cos_sim_f_flatten1=cos_sim_f_flatten[similarity_matrix_flatten==1]
cos_sim_f_flatten0=cos_sim_f_flatten[similarity_matrix_flatten==0]

File downloaded successfully to humap2_complexes_20200809.txt


100%|██████████| 2394/2394 [00:37<00:00, 64.04it/s] 


In [15]:
choice=np.random.choice(cos_sim_f_flatten1.shape[0], 2048)
cos_sim_f_flatten1=cos_sim_f_flatten1[choice]
cos_sim_f_flatten1=pd.DataFrame(cos_sim_f_flatten1,columns=['correlations'])
fig=px.violin(cos_sim_f_flatten1, y='correlations',width=500, height=400,title="SIMILARS")
fig.show()


choice=np.random.choice(cos_sim_f_flatten0.shape[0], 2048)
cos_sim_f_flatten0=cos_sim_f_flatten0[choice]
cos_sim_f_flatten0=pd.DataFrame(cos_sim_f_flatten0,columns=['correlations'])
fig=px.violin(cos_sim_f_flatten0, y='correlations',width=500, height=400,title="Not SIMILARS")
fig.show()

In [16]:
print("Not SIMILARS MEAN:",cos_sim_f_flatten0.mean())
print("SIMILARS MEAN:",cos_sim_f_flatten1.mean())

Not SIMILARS MEAN: correlations    0.031357
dtype: float32
SIMILARS MEAN: correlations    0.415202
dtype: float32


In [32]:
fig=px.scatter(df_to_be_shown,x='f0',y='f1',color='control',width=500, height=400)
fig.show()
fig=px.scatter(df_to_be_shown,x='f2',y='f3',color='control',width=500, height=400)
fig.show()
fig=px.scatter(df_to_be_shown,x='f4',y='f5',color='control',width=500, height=400)
fig.show()

c:\Users\khoda\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\khoda\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



c:\Users\khoda\AppData\Local\Programs\Python\Python310\lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.

